## Ⅰ. Einsum 이란?

특수한 표기법을 통해 다양한 텐서 연산을 수행할 수 있는 함수로 numpy, pytorch에서 다음과 같이 사용할 수 있다.
```python
np.einsum('ix,jx->ij', A, B)
torch.einsum('ix,jx->ij', [A, B])
```

## Ⅱ. Einsum 표기법

예를 들어 다음과 같은 식이 있을 때,

$$
C_{mqn} = \sum_t\sum_k A_{ntkm}B_{kq} = A_{ntkm}B_{kq}
$$

notation은 `'ntkm,kq->mqn'` 이다. 연산 방식은 2가지만 기억하면 된다.

1. Input에서의 겹치는 인덱스는 elementwise multiplication를 의미한다.

In [1]:
import numpy as np
import torch

a = [1,2,3]
b = [1,2,3]

np.einsum('i, i -> i', a, b)

array([1, 4, 9])

2. Output에 나오지 않은 인덱스는 그 축의 방향으로 summation이 이루어진다.

In [2]:
a = [1,2,3,4]
b = [1,2]

np.einsum('i, j -> i', a, b)

array([ 3,  6,  9, 12])

For문을 사용하여 표현하면 다음과 같이 Outer Loop 과 Inner Loop으로 나뉜다..

1. Free Indices : output에 나오는 인덱스 (m, q, n)
2. Summation Indices : 그 외 인덱스 (t, k)

In [3]:
A = np.arange(2*3*4*5).reshape(2, 3, 4, 5)
B = np.arange(4*7).reshape(4, 7)

N, T, K, M = A.shape
K, Q = B.shape

C = np.zeros((M, Q, N))

# Free indices
for m in range(M):
    for q in range(Q):
        for n in range(N):
            
            # Summation indices
            C[m, q, n] = np.sum([A[n, t, k, m] * B[k, q] for t in range(T) for k in range(K)])

Einsum을 사용하면 다음과 같다.

In [4]:
C_ = np.einsum('ntkm, kq -> mqn', A, B)

np.all(C_ == C)

True

## Ⅲ. Examples

### 1. Permutation

In [5]:
t = torch.Tensor(3, 5, 7)

torch.einsum('ijk -> kji', t).shape

torch.Size([7, 5, 3])

### 2. Sum

In [6]:
t = torch.Tensor([[1, 2],
                  [3, 4],
                  [5, 6]])

torch.einsum('ij -> ', t)

tensor(21.)

### 3. Column Sum

In [7]:
t = torch.Tensor([[1, 2],
                  [3, 4],
                  [5, 6]])

torch.einsum('ij -> j', t)

tensor([ 9., 12.])

### 4. Row Sum

In [8]:
t = torch.Tensor([[1, 2],
                  [3, 4],
                  [5, 6]])

torch.einsum('ij -> i', t)

tensor([ 3.,  7., 11.])

### 5. Matrix Vector Multiplication

In [9]:
A = torch.Tensor([[1, 2],
                  [3, 4]])
x = torch.Tensor([1, 2])

torch.einsum('ik, k -> i', A, x)

tensor([ 5., 11.])

### 6. Matrix Multiplication

In [10]:
A = torch.Tensor([[1, 2, 3],
                  [4, 5, 6]])
B = torch.Tensor([[1, 2],
                  [3, 4],
                  [5, 6]])

torch.einsum('ik, kj -> ij', A, B)

tensor([[22., 28.],
        [49., 64.]])

### 7. Dot Product

In [11]:
a = torch.Tensor([1, 2, 3])
b = torch.Tensor([1, 2, 3])

torch.einsum('i, i -> ', a, b)

tensor(14.)

### 8. Hadamard Product

In [12]:
A = torch.Tensor([[1, 2, 3],
                  [4, 5, 6]])
B = torch.Tensor([[1, 2, 3],
                  [4, 5, 6]])

torch.einsum('ij, ij -> ij', A, B)

tensor([[ 1.,  4.,  9.],
        [16., 25., 36.]])

### 9. Outer Product

In [13]:
a = torch.Tensor([0, 1, 2])
b = torch.Tensor([0, 1, 2, 3])

torch.einsum('i, j -> ji', a, b)

tensor([[0., 0., 0.],
        [0., 1., 2.],
        [0., 2., 4.],
        [0., 3., 6.]])

### 10. Batch Matrix Multiplication

In [14]:
A = torch.Tensor(4, 3, 2)
B = torch.Tensor(4, 2, 5)

torch.einsum('bik, bkj -> bij', A, B).shape

torch.Size([4, 3, 5])

### 11. Matrix Diagonal

In [15]:
A = torch.Tensor([[0, 1, 2],
                  [3, 4, 5],
                  [6, 7, 8]])

torch.einsum('ii -> i', A)

tensor([0., 4., 8.])

### 11. Trace

In [16]:
A = torch.Tensor([[0, 1, 2],
                  [3, 4, 5],
                  [6, 7, 8]])

torch.einsum('ii -> ', A)

tensor(12.)

## Ⅳ. Reference

https://rockt.github.io/2018/04/30/einsum